## Rules of the game:
- There are 100 prisoners in the prison
- In a room with 100 boxes, each box has a slip with a number between 1 and 100
- Each prisoner with a number between 1 and 100 goes into the room, opens a box and reads the slip. 
    - If the number on the slip is the prisoner's number, prisoner wins the round. Then all boxes are closed (but not shuffled!)
    - If the number doesn't match, prisoner can open another box. Each prisoner can try 50 boxes.
    - If the prisoner cannot find a match in 50 boxes, he/she fails the round.
- If they all succeed, they can all leave; but if one fails, they all lose!

Reference: Great video from Veritasium - https://www.youtube.com/watch?v=iSNsgj1OCLA

In [1]:
import numpy as np
import random as rn

#### Naive strategy: Random search

In [2]:
def random_search(boxes, ntrials, prisoner_id):
    box_order=np.arange(len(boxes))
    rn.shuffle(box_order) # create random box order for prisoner to follow during search
    for t in np.arange(ntrials):
        box_id=box_order[t]
        slip_value=boxes[box_id]
        if slip_value==prisoner_id: return True # this prisoner found his/her number!
    else: return False # this prisoner failed in ntrials!
    

#### Smart strategy: Prisoner starts with the box of his/her number and reads the slip. If the number on slip does not match, he/she switches to the box of slip value

In [3]:
def smart_search(boxes, ntrials, prisoner_id):
    box_id=prisoner_id
    for t in np.arange(ntrials):
        slip_value=boxes[box_id]
        if slip_value==prisoner_id: return True # this prisoner found his/her number!
        box_id=slip_value 
    else: return False # this prisoner failed in ntrials!

#### Game: Run of all prisoners and return False if any fails

In [4]:
def run_game(nboxes, ntrials, search_method):
    # create boxes with random numbers in them
    boxes=np.arange(nboxes)
    rn.shuffle(boxes)
    
    for p in np.arange(nboxes):
        is_success=search_method(boxes=boxes, ntrials=ntrials, prisoner_id=p)
        if not is_success: return False # if any prisoner fails, all of them fail
    else: return True # all prisoners have succeeded!

#### Conduct many runs over each strategies to compare success ratios 

In [5]:
nboxes=100
ntrials=50
nruns=10000
for search_method in [random_search, smart_search]:
    success_ratio=0
    for n in np.arange(nruns):
        result=run_game(nboxes=nboxes, ntrials=ntrials, search_method=search_method)
        success_ratio+=result/nruns
    print(f"{search_method.__name__} has a {success_ratio=}")

random_search has a success_ratio=0.0
smart_search has a success_ratio=0.31289999999998186


#### Observations:
- With random strategy, each prisoner has a 50% chance, therefore probability that all of them succeeds is (0.5)^100 which is an extremely small number
- With smart strategy, we observe ~31% success ratio!

In [6]:
from IPython.display import Image
Image(url='http://www.reactiongifs.com/r/but-why.gif')

## Explanation

One specific rule of the game gives the greatest clue: All prisoners have to succeed or they all fail. 

Veritasium explains this in great detail, but allow me as well: Let's say a prisoner 8 enters the room and applies the 'smart' strategy as follows:
1. Opens the 8th box and reads 52 on the slip
2. Opens the 52nd box and reads 78 on the slip
3. Opens the 78th box and reads 8 on the slip: The prisoner just succeeded!

So prisoner 8 opened boxes 8,52,78 respectively. Now, imagine the case: If prisoner 78 enters the room and follows the same strategy, he/she would open boxes 78,8,52. Same goes for prisoner 52 as he/she would follow 52,78,8 as well. That means boxes 8, 52 and 78 form a loop and since this loop is shorter than 50 elements, these 3 prisoners succeed with this strategy. If a prisoner, say prisoner 34 is in a loop > 50 elements, he/she will fail and make everyone else fail as well.

In other words, if boxes form loops where one of them have more than 50 elements, the smart strategy will fail. The probability of forming a loop of 100 elements is 1/100 (100!/100)/100!, a loop of 99 elements is 1/99 and so on. Therefore the proability of boxes forming a loop with greater than 50 elements is 1/50 + 1/51 + ... + 1/99 and 1/100.

In [7]:
prob_fail=sum([1/x for x in np.arange(50,100)+1])
prob_success=1-prob_fail
print(f'{prob_success=}, {prob_fail=}')

prob_success=0.3118278206898051, prob_fail=0.6881721793101949


### What if we let prisoners try 60 times?

In [8]:
prob_fail=sum([1/x for x in np.arange(60,100)+1])
prob_success=1-prob_fail
print(f'{prob_success=}, {prob_fail=}')

prob_success=0.4924928953121174, prob_fail=0.5075071046878826


In [9]:
nboxes=100
ntrials=60
nruns=10000
for search_method in [random_search, smart_search]:
    success_ratio=0
    for n in np.arange(nruns):
        result=run_game(nboxes=nboxes, ntrials=ntrials, search_method=search_method)
        success_ratio+=result/nruns
    print(f"{search_method.__name__} has a {success_ratio=}")

random_search has a success_ratio=0.0
smart_search has a success_ratio=0.48449999999996296
